# 1. Libraries

In [1]:
%load_ext autoreload
%autoreload 2
import discrimination
import pandas as pd
import pickle
import os
import numpy as np
import random
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, Dense, Activation, Flatten, Dropout
from keras.models import Sequential
from keras import regularizers
import matplotlib.pyplot as plt
import itertools
import tqdm
import seaborn

INFO: {'User-Agent': 'Opera/9.80 (X11; Linux i686; Ubuntu/14.10) Presto/2.12.388 Version/12.16'}
Using Theano backend.


# 2. Data Preperation

## Create lists containg all the *English* texts, clean them up, etc., and finally save them.

Download from Mongo all the texts collected (tables are named diary.com, my-diary.org and everydaysexism) and create lists with them, one for each table.

In [ ]:
texts_diary=[]
texts_mydiary = []
texts_everydaysexism = []

table = discrimination.mongo.collection(collection= "diary.com")
for x in table.find():
    texts_diary.append(x['text'])

table = discrimination.mongo.collection(collection= "my-diary.org")
for x in table.find():
    texts_mydiary.append(x['text'])

table = discrimination.mongo.collection(collection= "everydaysexism")
for x in table.find():
    texts_everydaysexism.append(x['text'])

In [ ]:
print(len(texts_diary), "texts collected from diary.com")
print(len(texts_mydiary), "texts collected from my-diary.org")
print(len(texts_everydaysexism), "texts collected from everydaysexism.com")

From each list of texts, keep only the ones in English. This process takes a long time... a timer is printed every 10,000 texts (by default).

In [ ]:
texts_diary = discrimination.texts.keep_english(texts_diary)
texts_mydiary = discrimination.texts.keep_english(texts_mydiary)
texts_everydaysexism = discrimination.texts.keep_english(texts_everydaysexism)

Clean the texts from urls, html, hastags; then make them lowercase.

In [ ]:
texts_diary = discrimination.texts.clean(texts_diary)
texts_mydiary = discrimination.texts.clean(texts_mydiary)
texts_everydaysexism = discrimination.texts.clean(texts_everydaysexism)

texts_diary = discrimination.texts.lowercase(texts_diary)
texts_mydiary = discrimination.texts.lowercase(texts_mydiary)
texts_everydaysexism = discrimination.texts.lowercase(texts_everydaysexism)

Save the text-lists using Pickle to avoid redoing this.

In [ ]:
# Save
# pickle.dump(texts_diary, open("pickles/texts_diary.p", "wb"))
# pickle.dump(texts_mydiary, open("pickles/texts_mydiary.p", "wb"))
# pickle.dump(texts_everydaysexism, open("pickles/texts_everydaysexism.p", "wb"))

# Load
texts_diary = pickle.load(open("pickles/texts_diary.p", "rb"))
texts_mydiary = pickle.load(open("pickles/texts_mydiary.p", "rb"))
texts_everydaysexism = pickle.load(open("pickles/texts_everydaysexism.p", "rb"))

In [ ]:
print(len(texts_diary), "texts collected from diary.com after cleaning")
print(len(texts_mydiary), "texts collected from my-diary.org after cleaning")
print(len(texts_everydaysexism), "texts collected from everydaysexism.com after cleaning")

## Split my-diary.org texts in sentences

This step is done in order to split up the non-sexist texts from my-diary.org (since they are too long) in order to match the length of the sexist texts from everydaysexism. This greatly increases the number of texts obtained from my-diary.org (roughly by 10 times).

In [ ]:
sentences_diary = discrimination.texts.sentences_split(texts_diary)
sentences_mydiary = discrimination.texts.sentences_split(texts_mydiary)
sentences_everydaysexism = discrimination.texts.sentences_split(texts_everydaysexism)

In [ ]:
list_of_texts = [sentences_everydaysexism, sentences_diary, sentences_mydiary]
legend = ["Everydaysexism.com", "Diary.com", "My-diary.org"]
discrimination.texts.sentences_plot(list_of_texts, (0,0.27), 30, [10,5], 90, legend)

Looks like texts from everydaysexism.com and diary.com have comparable lengths in terms of sentences. But my-diary.org texts seem way longer. Let's take a closer look.

In [ ]:
discrimination.texts.sentences_plot(list_of_texts, (0,0.032), 50, [10,5], 90, legend)

Texts from everydaysexism.com and diary.com have way less sentences than my-diary.org. Use a hard limit and split all texts of my-diary.org until they are up to 20 sentences long at most.

In [ ]:
texts = []
for sentences in sentences_mydiary:
    division = round( len(sentences) / 20 )
    for i in range(division + 1):
        text = "".join(sentences[20*i : 20*(i+1)])
        if len(text) >= 20:
            texts.append(text)
            
sentences_mydiary2 = discrimination.texts.sentences_split(texts)

In [ ]:
list_of_texts = [sentences_everydaysexism, sentences_diary, sentences_mydiary2]
legend = ["Everydaysexism.com", "Diary.com", "My-diary.org (first split)"]
discrimination.texts.sentences_plot(list_of_texts, (0,0.5), 30, [10,5], 90, legend)

This has created obviously a very large spike at 20 sentences (~45%) of the texts. What we can do is take a normal distribution with $\mu=2$ and $\sigma = 6$. Then we will discard non-positive values and round all other values to the closest integer. This draw will -almost- be a half-normal with $\mu=2$ and $\sigma = 6$. It will be able to reproduce, to some extent, the distribution of everydaysexism.com texts, where many texts have just one or two sentences, and will also have all 99.6% of texts having between 1 and $\mu + 3\times\sigma = 20$ sentences. (It won't however look like a half-normal cause when splitting $x$, the number of sentnces in $x_1$ follows this distribution, but the number of sentences left in $x_2 = x -x_1$ does not).

...well after a little tweaking, setting $\mu=4$ and $\sigma=7$ makes the distribution resemble the everyday sexism one. So I'll settle for this.

In [ ]:
texts.clear()

for sentences in sentences_mydiary2:   
    draw = -1
    while draw <= 0:
        draw = int(round(random.normalvariate(4,7),0))
    
    division = round( len(sentences) / draw )
    
    for i in range(division + 1):   
        text = "".join(sentences[draw*i : draw*(i+1)])
        if len(text) >= 20:
            texts.append(text)

sentences_mydiary3 = discrimination.texts.sentences_split(texts)

In [ ]:
list_of_texts = [sentences_everydaysexism, sentences_diary, sentences_mydiary3]
legend = ["Everydaysexism.com", "Diary.com", "My-diary.org (second split)"]
discrimination.texts.sentences_plot(list_of_texts, (0,0.5), 30, [10,5], 90, legend)

This looks good. Make "texts_mydiary" equal to the final "texts" variable and move on.

In [ ]:
texts_mydiary = texts

In [ ]:
print("37K of English texts collected resulted in", len(texts_mydiary), "after splitting")

## Tokenize texts<sup>1</sup>, spell-check each word<sup>2</sup>, remove stop-words again, discard the empty resulting tokens and the ones with just one word. Save.

<sup>1</sup> Tokens only contain letters, words with letters and numbers are discarded.  
<sup>2</sup> If the word does not exist in the dictionary, then all existing words that are 1 letter "change" away are found. For example, from the words *cta*, *cet*, *ca*, *catt*, the word *cat* is respectively one neighbouring letter-swap, letter-change, letter addition and letter removal away. After all such words are found in the dictionary, the most frequent one is chosen to substitute the misspelled word.

In [ ]:
# Tokenize texts and remove stop-words
tokens_diary = discrimination.texts.tokenize(texts_diary)
tokens_mydiary = discrimination.texts.tokenize(texts_mydiary)
tokens_everydaysexism = discrimination.texts.tokenize(texts_everydaysexism)

# Spell-check tokens. This actually takes some time (not too much) so there's a timer every 20.000 tokens checked.
tokens_diary = discrimination.texts.spellcheck_tokens(tokens_diary)
tokens_mydiary = discrimination.texts.spellcheck_tokens(tokens_mydiary)
tokens_everydaysexism = discrimination.texts.spellcheck_tokens(tokens_everydaysexism)

# Remove stop-words a second time, in case some stopwords where misspelled.
tokens_diary = discrimination.texts.remove_stopwords(tokens_diary)
tokens_mydiary = discrimination.texts.remove_stopwords(tokens_mydiary)
tokens_everydaysexism = discrimination.texts.remove_stopwords(tokens_everydaysexism)

# Before discarding some tokens, keep a copy of the original ones.
tokens_diary_all = tokens_diary.copy()
tokens_mydiary_all = tokens_mydiary.copy()
tokens_everydaysexism_all = tokens_everydaysexism.copy()

# Discard tokens with 0 or 1 words.
for tokens in [tokens_diary, tokens_mydiary, tokens_everydaysexism]:
    temp = tokens.copy()
    tokens.clear()
    for token in temp:
        if len(token) > 1:
            tokens.append(token)

In [ ]:
# Save
# pickle.dump(tokens_diary, open("pickles/tokens_diary.p", "wb"))
# pickle.dump(tokens_mydiary, open("pickles/tokens_mydiary.p", "wb"))
# pickle.dump(tokens_everydaysexism, open("pickles/tokens_everydaysexism.p", "wb"))
# pickle.dump(tokens_diary_all, open("pickles/tokens_diary_all.p", "wb"))
# pickle.dump(tokens_mydiary_all, open("pickles/tokens_mydiary_all.p", "wb"))
# pickle.dump(tokens_everydaysexism_all, open("pickles/tokens_everydaysexism_all.p", "wb"))

# Load
tokens_diary = pickle.load(open("pickles/tokens_diary.p", "rb"))
tokens_mydiary = pickle.load(open("pickles/tokens_mydiary.p", "rb"))
tokens_everydaysexism = pickle.load(open("pickles/tokens_everydaysexism.p", "rb"))
tokens_diary_all = pickle.load(open("pickles/tokens_diary_all.p", "rb"))
tokens_mydiary_all = pickle.load(open("pickles/tokens_mydiary_all.p", "rb"))
tokens_everydaysexism_all = pickle.load(open("pickles/tokens_everydaysexism_all.p", "rb"))

## Convert tokens back to text (so Keras can be happy) and labels for these texts. Keep a record of original texts, tokens, tokens-to-texts, labels. Save.

The record is needed so that later on we can check the prediction for tokens of interest (e.g. the borderline sexist ones) and easily access the original text.

In [ ]:
# Convert tokens back to text for Keras
texts_keras = []
for token in itertools.chain(tokens_diary, tokens_mydiary, tokens_everydaysexism):
    text = ""
    for word in token:
        text += word + " "        
    texts_keras.append(text)

# Create labels
labels_keras = np.zeros(len(texts_keras))
labels_keras[-len(tokens_everydaysexism):] = 1

# Record texts, tokens
record = []
for text, token in zip(texts_diary, tokens_diary_all):
    record.append({"text": text, "token": token, "truth": 0})
for text, token in zip(texts_mydiary, tokens_mydiary_all):
    record.append({"text": text, "token": token, "truth": 0})
for text, token in zip(texts_everydaysexism, tokens_everydaysexism_all):
    record.append({"text": text, "token": token, "truth": 1})

# Add the tokens-to-text in the record
for entry in record:
    token_to_text = ""
    for word in entry["token"]:
        token_to_text += word + " "        
    entry["token_to_text"] = token_to_text

In [2]:
# Save
# pickle.dump(record, open("pickles/record.p", "wb"))
# pickle.dump(texts_keras, open("pickles/texts_keras.p", "wb"))
# pickle.dump(labels_keras, open("pickles/labels_keras.p", "wb"))

# Load
record = pickle.load(open("pickles/record.p", "rb"))
texts_keras = pickle.load(open("pickles/texts_keras.p", "rb"))
labels_keras = pickle.load(open("pickles/labels_keras.p", "rb"))

In [ ]:
print(len(labels_keras) - len(tokens_everydaysexism), "non-sexist tokens and", len(tokens_everydaysexism), "sexist ones.")

# 2. Neural Network in Keras

## Data\Labels preparation and save. Parse the word embeddings, create the embedding layer.

In [3]:
# Tokenizing - Sequencing
tokenizer = Tokenizer(lower = False)
tokenizer.fit_on_texts(texts_keras)
sequences = tokenizer.texts_to_sequences(texts_keras)
word_index = tokenizer.word_index

# Create and shuffle data and labels
data = pad_sequences(sequences, maxlen=256)
labels = np.asarray(labels_keras)
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

# Save data and labels
pickle.dump(data, open("pickles/data.p", "wb"))
pickle.dump(labels, open("pickles/labels.p", "wb"))

# Split 80-20
nb_validation_samples = int(0.2 * data.shape[0])
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

In [4]:
# Parse the GloVe word embeddings and save
glove_dir = "glove/"
embeddings_index = {}
f = open(os.path.join(glove_dir, "glove.42B.300d.txt"))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype="float32")
    embeddings_index[word] = coefs
f.close()

In [5]:
# Create the embedding matrix
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
# Delete the embeddings index as it's no longer needed.
del embeddings_index
# Create the embedding layer
embedding_layer = Embedding(len(word_index) + 1, 300, input_length=256,
                            weights=[embedding_matrix],
                            trainable=False)

## Network setup, compilation, results, testing. Save model weights.

In [6]:
# Setup
model = Sequential()
model.add(embedding_layer)
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(256, activation="relu", kernel_regularizer = regularizers.l2(0.001)))
model.add(Dense(16, activation="relu", kernel_regularizer = regularizers.l2(0.001)))
model.add(Dense(1, activation="sigmoid"))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 256, 300)          26588700  
_________________________________________________________________
flatten_1 (Flatten)          (None, 76800)             0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 76800)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               19661056  
_________________________________________________________________
dense_2 (Dense)              (None, 16)                4112      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 46,253,885
Trainable params: 19,665,185
Non-trainable params: 26,588,700
_________________________________

In [7]:
# Compilation
model.compile(optimizer = "Adam",
              loss = "binary_crossentropy",
              metrics = ["acc"])
history = model.fit(x_train, y_train,
                    epochs = 3,
                    batch_size = 256,
                    validation_data = (x_val, y_val))

# Save model weights
model.save_weights("pickles/model2.h5")

Train on 348965 samples, validate on 87241 samples
Epoch 1/3
348965/348965 [==============================] - 1178s 3ms/step - loss: 0.3975 - acc: 0.8993 - val_loss: 0.3453 - val_acc: 0.9252
Epoch 2/3
348965/348965 [==============================] - 3916s 11ms/step - loss: 0.3367 - acc: 0.9111 - val_loss: 0.2889 - val_acc: 0.9280
Epoch 3/3
348965/348965 [==============================] - 4325s 12ms/step - loss: 0.3167 - acc: 0.9146 - val_loss: 0.2808 - val_acc: 0.9281


In [ ]:
# Make some graphs
acc = history.history["acc"]
val_acc = history.history["val_acc"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]

epochs = range(1, len(acc) + 1)

plt.figure(figsize = [12,6])

# Accuracy graph
plt.subplot(1, 2, 1)
plt.plot(epochs, acc, "bo", label = "Training acc")
plt.plot(epochs, val_acc, "b", label = "Validation acc")
plt.title("Training and validation accuracy")
plt.ylim([0,1])
plt.legend()
# Loss graph
plt.subplot(1, 2, 2)
plt.plot(epochs, loss, "bo", label = "Training loss")
plt.plot(epochs, val_loss, "b", label = "Validation loss")
plt.title("Training and validation loss")
plt.ylim([0,1])
plt.legend()

plt.show()

## Test the model!

In [28]:
# Test the network
test = [""]

# Convert the test phrase to lowercase, tokenize, spellcheck, remove stopwords. 
test = discrimination.texts.lowercase(test)
test = discrimination.texts.tokenize(test)
test = discrimination.texts.spellcheck_tokens(test)
test = discrimination.texts.remove_stopwords(test)

# Convert the token back to text, sequence it, pad it, feed it into the model.
text = ""
for item in test:
    for word in item:
        text += word + " "   
test_sequence = tokenizer.texts_to_sequences([text])
print(test)
x_test = pad_sequences(test_sequence, maxlen=256)
model.load_weights("pickles/model2.h5")
# Make the output look pretty... because it deserves it.
str(round(model.predict(x_test)[0,0]*100,0))[:-2] + "% sexist"

[['rape', 'woman']]


'63% sexist'

# 3. Borderline results

In [ ]:
# Get all model predictions
model.load_weights("pickles/model2.h5")
predictions = model.predict(data)

In [ ]:
# Define the borderline
borderline = [0.49, 0.51]

# Find all tokens that are on the borderline
borderline_tokens = []
for i, prediction in enumerate(predictions):
    if borderline[0] <= prediction[0] <= borderline[1]: 
        token = texts_keras[i]
        sexist = labels_keras[i]
        borderline_tokens.append((token, prediction[0], sexist))

# Create a list with all the borderline texts
borderline_texts = []
for token in borderline_tokens:
    if token[2]==1:
        for entry in record:
            if entry["token_to_text"] == token[0]:
                borderline_texts.append({"text": entry["text"], "prediction": token[1], "truth": 1})
    else:
        for entry in record:
            if entry["token_to_text"] == token[0]:
                borderline_texts.append({"text": entry["text"], "prediction": token[1], "truth": 0})
                
# Make the list into a DataFrame
df_borderline = pd.DataFrame(borderline_texts, columns = ["text", "prediction", "truth"])

In [ ]:
# Save
# pickle.dump(df_borderline, open("pickles/df_borderline.p", "wb"))

# Load
df_borderline = pickle.load(open("pickles/df_borderline.p", "rb"))

In [ ]:
pd.set_option('display.max_colwidth', 0)
df_borderline[df_borderline["truth"]==1].head(3)

# 4. Word frequencies

## Retrieve the common vocabulary (sexist & non-sexist texts). Order it by frequency for each type of texts. Save.

In [ ]:
# Temp variables
temp_list = []
temp_sexist = []
temp_nonsexist = []

# Retrieve sexist vocabulary and convert tokens into sets
sexist_vocabulary = set()
for token in tokens_everydaysexism: 
    x = set(token)
    sexist_vocabulary |= x
    temp_sexist.append(x)

# Retrieve non-sexist vocabulary and convert tokens into sets
nonsexist_vocabulary = set()
for token in itertools.chain(tokens_diary, tokens_mydiary):
    x = set(token)
    nonsexist_vocabulary |= x
    temp_nonsexist.append(x)
    
# Retrieve common vocabulary
common_vocabulary = sexist_vocabulary & nonsexist_vocabulary

I measure frequency by $\frac{\text{number of tokens word appears in}}{\text{number of tokens}}$. This is done separately for each word in the sexist and non-sexist texts so that comparisons can be made. This takes some time (around 30 minutes).

In [ ]:
# Frequency denominators
denominator_sexist = len(temp_sexist)
denominator_nonsexist = len(temp_nonsexist)

# Frequency numerators
for word in tqdm.tqdm(common_vocabulary):
    timer += 1
    numerator_sexist = 0
    numerator_nonsexist = 0
    
    for token in temp_sexist:
        if word in token:
            token -= set(word)
            numerator_sexist += 1
            
    for token in temp_nonsexist:
        if word in token:
            token -= set(word)
            numerator_nonsexist += 1
    
    # Calculate word frequencies
    pct_sexist = numerator_sexist / denominator_sexist
    pct_nonsexist = numerator_nonsexist / denominator_nonsexist
    
    # Add info to the temp list
    temp_list.append({'word': word, 'pct_sexist': pct_sexist, 'pct_nonsexist': pct_nonsexist})
        
# Make the list into a DataFrame
df_word_rank = pd.DataFrame(temp_list, columns = ["word", "pct_sexist", "pct_nonsexist"])

In [ ]:
# Save
# pickle.dump(df_word_rank, open("pickles/df_word_rank.p", "wb"))

# Load
df_word_rank = pickle.load(open("pickles/df_word_rank.p", "rb"))

## Using the word frequencies, create a dataframe so that the top-x most common words in sexist texts can be retrieved and compared with their non-sexist rank. Also do the opposite, i.e. top-x most common words in non-sexist texts can be retrieved and compared with their sexist rank.

In [ ]:
# Show percentages as integers from 0 to 100.
df_word_rank["pct_sexist"] = round(df_word_rank["pct_sexist"] * 100, 1)
df_word_rank["pct_nonsexist"] = round(df_word_rank["pct_nonsexist"] * 100, 1)
# Create NonSexist Rank
df_word_rank = df_word_rank.sort_values(by="pct_nonsexist", ascending = False)
df_word_rank = df_word_rank.reset_index(drop=True)
df_word_rank["nonsexist_rank"] = 1 + df_word_rank.index
# Create Sexist Rank
df_word_rank = df_word_rank.sort_values(by="pct_sexist", ascending = False)
df_word_rank = df_word_rank.reset_index(drop=True)
df_word_rank["sexist_rank"] = 1 + df_word_rank.index
# Reorder columns
df_word_rank = df_word_rank[["word", "pct_sexist", "sexist_rank", "nonsexist_rank", "pct_nonsexist"]]
# Add Rank Difference, Absolute Rank Difference, and Rank Difference Squared columns
df_word_rank["abs_rank_diff"] = abs( df_word_rank["nonsexist_rank"] - df_word_rank["sexist_rank"] )
df_word_rank["sq_rank_diff"] = df_word_rank["abs_rank_diff"] ** 2

In [ ]:
# Interested in Top-X, where...
X = 20

# Print Statistics
print("Absolute Rank Difference (Sexist vs Non-Sexist) of top-100 words is " + str(df_word_rank["abs_rank_diff"][:X].mean()))
print("Rank Difference Squared (Sexist vs Non-Sexist) of top-100 words is " + str(df_word_rank["sq_rank_diff"][:X].mean()))

# Display the dataframe
pd.set_option('display.max_rows', 50)
df_word_rank.head(X)

In [ ]:
# Order by NonSexist Rank
df_word_rank = df_word_rank.sort_values(by="pct_nonsexist", ascending = False)
df_word_rank = df_word_rank.reset_index(drop=True)

In [ ]:
# Print Statistics
print("Absolute Rank Difference (Non-Sexist vs Sexist) of top-100 words is " + str(df_word_rank["abs_rank_diff"][:X].mean()))
print("Rank Difference Squared (Non-Sexist vs Sexist) of top-100 words is " + str(df_word_rank["sq_rank_diff"][:X].mean()))

# Display the dataframe
df_word_rank.head(X)

# 5. Confusion Matrix and distribution of scores

## Confusion matrix

In [ ]:
# Create a predicted labels list
labels_predicted = []
for prediction in predictions:
    labels_predicted.append( round(prediction[0]) )
# Calculate the confusion matrix
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score
CF = confusion_matrix(labels, labels_predicted)
#"Disentangle" the matrix
TN = round((CF[0,0] / sum(CF[0,:])) * 100, 1)
FN = round((CF[0,1] / sum(CF[0,:])) * 100, 1)
TP = round((CF[1,1] / sum(CF[1,:])) * 100, 1)
FP = round((CF[1,0] / sum(CF[1,:])) * 100, 1)
GTN = round((CF[0,0] / (sum(CF[0,:]) + sum(CF[1,:]))) * 100, 1)
GFN = round((CF[0,1] / (sum(CF[0,:]) + sum(CF[1,:]))) * 100, 1)
GTP = round((CF[1,1] / (sum(CF[0,:]) + sum(CF[1,:]))) * 100, 1)
GFP = round((CF[1,0] / (sum(CF[0,:]) + sum(CF[1,:]))) * 100, 1)
# Print the results
print("True positives account for "+str(TP)+"% or "+str(GTP)+"% of the total (sexist texts labelled as sexist).")
print("True negatives account for "+str(TN)+"% or "+str(GTN)+"% of the total (non-sexist texts labelled as non-sexist).")
print("False positives account for "+str(FP)+"% or "+str(GFP)+"% of the total (sexist texts labelled as non-sexist).")
print("False negatives account for "+str(FN)+"% or "+str(GFN)+"% of the total (non-sexist texts labelled as sexist).")

## Distribution of scores

In [ ]:
# Record all predictions (x) but also seperately for each class (x_sexist and x_nonsexist).
x = []
x_sexist = []
x_nonsexist = []
for i, p in enumerate(predictions):
    x.append(p[0])
    if labels[i] == 1:
        x_sexist.append(p[0])
    else:
        x_nonsexist.append(p[0])

# Transform predictions in the 0-100 range, and round them up to the nearast integer.        
x_100 = []
x_sexist_100 = []
x_nonsexist_100 = []
for item in x:
    x_100.append(round(item*100))
for item in x_sexist:
    x_sexist_100.append(round(item*100))
for item in x_nonsexist:
    x_nonsexist_100.append(round(item*100))

# Count the (rounded) predictions and transform to percentages.
y = []
y_sexist = []
y_nonsexist = []
for i in range(101):
       y.append(100*x_100.count(i)/len(x_100))
for i in range(101):
       y_sexist.append(100*x_sexist_100.count(i)/len(x_sexist_100))
for i in range(101):
       y_nonsexist.append(100*x_nonsexist_100.count(i)/len(x_nonsexist_100))

In [ ]:
plt.figure(figsize=[14,4], dpi=110)
plt.subplot(1,2,1)
plt.ylim([0,20])
plt.plot(range(101), y)
plt.xlabel("Probability of sexist")
plt.ylabel("Percentage of texts")

plt.subplot(1,2,2)
plt.ylim([0,20])
plt.plot(range(101), y_sexist)
plt.plot(range(101), y_nonsexist)
plt.legend(["Sexist", "Non-Sexist"])
plt.xlabel("Probability of sexist");

Zoom-in a little bit...

In [ ]:
plt.figure(figsize=[14,4], dpi=110)
plt.subplot(1,2,1)
plt.ylim([0,3])
plt.plot(range(101), y)
plt.xlabel("Probability of sexist")
plt.ylabel("Percentage of texts")

plt.subplot(1,2,2)
plt.ylim([0,3])
plt.plot(range(101), y_sexist)
plt.plot(range(101), y_nonsexist)
plt.legend(["Sexist", "Non-Sexist"])
plt.xlabel("Probability of sexist");